In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 26 14:40:07 2026

@author: ebbaelvelid
"""

import numpy as np
import os

def clear_console():
    os.system('clear')

t_sol = np.array([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])
f_sol = np.array([12.00, 15.10, 19.01, 23.92, 30.11, 37.90, 47.70, 60.03, 75.56])

def trapets(f, n, limits):
    """
    Från modul 4 kod
    Approximerar integral med trapetser
    Noggrannhetsordning p=2 -> fel proportionellt mot h²
    """
    a, b = limits
    h = (b - a) / n
    
    if callable(f): # om f är en funktion: sampla den
        x = np.linspace(a, b, n+1)
        y = f(x)
    else: # om f är en array: använd värdena direkt
        y = f
    
    I = h * (y[0] + 2*np.sum(y[1:-1]) + y[-1]) / 2
    return I

def uppgift3a_b():
    print("\n3a) Trapetsregeln")
    # Approximerar kurva med räta linjer
    # I = h/2 · (f₀ + 2f₁ + 2f₂ + ... + 2fn₋₁ + fn)
    
    # Test med integral
    test_f = lambda x: x**3 * np.exp(x)
    I_exact = 6 + 2*np.exp(2)
    I_test = trapets(test_f, 100, [0, 2])
    # ∫₀² x³eˣ dx")
    print(f"Exakt: {I_exact:.6f}")
    print(f"Trapets (n=100): {I_test:.6f}")
    print(f"Fel: {abs(I_exact - I_test):.6f}")
    
    print("\n3b) Konvergensstudie av a)") # och noggrannhetsordning (med steglängdshalvering)
    # Fel = h² så halvera h blir fel/4
    print(f"\n{'n':>5} {'h':>8} {'I(h)':>12} {'Fel':>12} {'Kvot':>8}")
    
    errors = []
    for n in [10, 20, 40, 80]:
        I_n = trapets(test_f, n, [0, 2])
        e = abs(I_exact - I_n)
        errors.append(e)
        h = 2.0/n
        if len(errors) > 1:
            kvot = errors[-2]/errors[-1]
            print(f"{n:5d} {h:8.5f} {I_n:12.6f} {e:12.2e} {kvot:8.2f}")
        else:
            print(f"{n:5d} {h:8.5f} {I_n:12.6f} {e:12.2e}     -")
    # p = ca log2(e_h / e_{h/2}) blir ca = 2 för trapetsregeln
    p = np.mean([np.log2(errors[i]/errors[i+1]) for i in range(len(errors)-1)])
    print(f"\nNoggrannhetsordning p ≈ {p:.2f}") # blir ca 2
        
def uppgift3c_d():
    print("\n3c) Energi 2014-2022") # diskret data
    I_total = trapets(f_sol, len(f_sol)-1, [2014, 2022]) # använder samma trapets som tidigare
    print(f"Total energi: {I_total:.2f} kWår")
    
    print("\n3d) Konvergensstudie av c)")
    # Ingen exakt lösning finns så fel approximeras via skillnad mellan två stegstorlekar
    # Uppskatta fel: e_2h ungefär = |I_h - I_2h|
    
    I_vals = []
    for h in [1, 2, 4]:
        idx = range(0, len(t_sol), h)
        I_h = trapets(f_sol[idx], len(idx)-1, [t_sol[idx[0]], t_sol[idx[-1]]])
        I_vals.append(I_h)
        print(f"h={h}: I={I_h:.2f} kWår")
    
    if len(I_vals) >= 3:
        p = np.log2(abs(I_vals[2] - I_vals[1]) / abs(I_vals[1] - I_vals[0]))
        print(f"Uppskattad p ≈ {p:.2f}")

def uppgift3e():
    print("\n3e) Richardson och Simpsons")
    # Richardson: Kombinera T_h och T_2h för bättre resultat
    # Formel: I_rich = T_h + (T_h - T_2h)/3
    
    I_h1 = trapets(f_sol, len(f_sol)-1, [2014, 2022])
    
    idx = range(0, len(f_sol), 2)
    I_h2 = trapets(f_sol[idx], len(idx)-1, [t_sol[idx[0]], t_sol[idx[-1]]])
    # Trapetsregeln har p = 2 så Richardson-faktor = 1 / (2^p - 1) = 1/3
    I_rich = I_h1 + (I_h1 - I_h2) / 3
    print(f"Richardson: {I_rich:.2f} kWår")
    
    # Simpsons regel: andragradspolynom över två intervall
    # I = h/3 · (f₀ + 4f₁ + 2f₂ + 4f₃ + ... + fn)
    h = 1
    I_simp = h * (f_sol[0] + 4*np.sum(f_sol[1:-1:2]) + 
                  2*np.sum(f_sol[2:-2:2]) + f_sol[-1]) / 3
    print(f"Simpson: {I_simp:.2f} kWår") # h=1
    print(f"Skillnad: {abs(I_rich - I_simp):.4f}") # bör vara 0
    # Simpsons regel härleds via Richardsonextrapolation av trapetsregeln

def uppgift3f_g():
    print("\n3f) Exponentiell modell f(t) = ae^(b(t-2014))")
    
    t_shift = t_sol - 2014
    ln_f = np.log(f_sol)
    
    # Skapar matris för minstakvadratmetoden
    # ln(f) = ln(a) + b*t
    A = np.column_stack([np.ones(len(t_shift)), t_shift])
    # Lös minstakvadratproblemet
    c = np.linalg.lstsq(A, ln_f, rcond=None)[0]
    
    a = np.exp(c[0])
    b = c[1]
    print(f"a = {a:.4f}, b = {b:.4f}")
    print("\n3g) Projektbedömning")
    f_2023 = a * np.exp(b * 9)
    print(f"Predikterad effekt 2023: {f_2023:.2f} kW")
    
    t_ext = np.append(t_sol, 2023)
    f_ext = np.append(f_sol, f_2023)
    
    I_2023 = trapets(f_ext, len(f_ext)-1, [t_ext[0], t_ext[-1]])
    print(f"Total energi 2014-2023: {I_2023:.2f} kWår")
    
    print(f"\nVillkor 1 (Effekt > 100 kW): {f_2023:.2f} > 100? {'Ja' if f_2023 > 100 else 'Nej'}")
    print(f"Villkor 2 (Energi > 350 kWår): {I_2023:.2f} > 350? {'Ja' if I_2023 > 350 else 'Nej'}")
    
    if f_2023 > 100 or I_2023 > 350:
        print("\n Pilotprojekt lyckat") # ja pga ett av vilkoren uppfyllda

def main():
    clear_console()
    uppgift3a_b()
    uppgift3c_d()
    uppgift3e()
    uppgift3f_g()
    

if __name__ == "__main__":
    main()


3a) Trapetsregeln
Exakt: 20.778112
Trapets (n=100): 20.783038
Fel: 0.004926

3b) Konvergensstudie av a)

    n        h         I(h)          Fel     Kvot
   10  0.20000    21.269321     4.91e-01     -
   20  0.10000    20.901176     1.23e-01     3.99
   40  0.05000    20.808894     3.08e-02     4.00
   80  0.02500    20.785809     7.70e-03     4.00

Noggrannhetsordning p ≈ 2.00

3c) Energi 2014-2022
Total energi: 277.55 kWår

3d) Konvergensstudie av c)
h=1: I=277.55 kWår
h=2: I=281.20 kWår
h=4: I=295.56 kWår
Uppskattad p ≈ 1.98

3e) Richardson och Simpsons
Richardson: 276.33 kWår
Simpson: 276.33 kWår
Skillnad: 0.0000

3f) Exponentiell modell f(t) = ae^(b(t-2014))
a = 11.9990, b = 0.2300

3g) Projektbedömning
Predikterad effekt 2023: 95.10 kW
Total energi 2014-2023: 362.88 kWår

Villkor 1 (Effekt > 100 kW): 95.10 > 100? Nej
Villkor 2 (Energi > 350 kWår): 362.88 > 350? Ja

 Pilotprojekt lyckat
